# DCGAN: Deep Convolutional Generative Adversarial Network

In this notebook, you will reproduce the results obtained in the famous paper [Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks](https://arxiv.org/abs/1511.06434).

# 1. Setting up the environment
We import here all packages you will need throughout this notebook. Notice that we will be using the `LeakyReLU` activation function for the discriminator, but since it is not part of the standard activations functions, we will find it in `keras.layers.advanced_activations`. We will use a smaller version of the network defined in the paper, because we aim at generating MNIST-like images.

In [ ]:
from __future__ import print_function, division

from keras import backend as K
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
%matplotlib inline

import sys

import numpy as np

# 2. The data: MNIST dataset
To train the discriminator, we need real images. We will use MNIST images (but you could also use the German Traffic Sign dataset, if you're feeling brave!). Notice that it is sufficient to use the training set images, thus we use the underscore symbol `_` to state that we ignore all other outputs of the function `mnist.load_data()`. As usual, fill the blank variables!

In [ ]:
# Load the dataset
(X_train, _), (_, _) = mnist.load_data()

# TODO assign the correct values to the following variables!
n_train_samples =
img_rows =
img_cols =
# TODO this time, we also explicitly define the number of channels
n_channels = 

print("Training dataset: {} images. Size {}x{} {}-channel pixels".format(n_train_samples, img_rows, img_cols, n_channels))

# 3. Preprocessing the data
It has been empirically found that it is better to scale the pixel values to the $[-1, 1]$ range. Apply the correct transformation to the `X_train` samples!

In [ ]:
# TODO Rescale data to the [-1, 1] range. NOT to [0,1]!
X_train = 

Let's define the shape of the input of our discriminator network. This is the shape of one MNIST image.

In [ ]:
#TODO assign the right value to input_shape
if K.image_data_format() == 'channels_first':
    input_shape = 
else:
    input_shape = 


X_train = X_train.reshape(n_train_samples, input_shape[0], input_shape[1], input_shape[2])

# 4. Visualizing the data
OK, unless you are using a new dataset, you can skip this step!

# 5. Applying Neural Networks to the problem
Let us recap what the two neural networks will do:

- the **generator** takes an array of random numbers as input and returns an image of the same size of those contained in the training set. The size of the random number array is called the size of the latent dimension or, in our code `latent_dim`: we set it to 100.
- the **discriminator** takes an image as input and the probability of the image being real, with `0` (`1`) meaning the image is classified as fake (real) with absolute certainty.

In [ ]:
latent_dim = 100

Next, we need to define a function which we will use to build the discriminator. The discriminator is a simple CNN which takes an image as input and outputs a single number. Fill the missing variables, and notice that, since we are using an advanced activation function, we need to create a layer holding it.

In [ ]:
#TODO define this single missing variable!
output_dim = 

def build_discriminator():

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=input_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(output_dim, activation='sigmoid'))

    model.summary()

    return model

Now, we build the generator. The generator starts from an array of size `latent_dim` and outputs an image of the correct size (equal to the size of the real images) as output. Notice that we use a `UpSampling2D` layer, which you might want to lookup in the [documentation](https://keras.io/layers/convolutional/#upsampling2d)!

Compare this generator to the one described in the [original paper](https://arxiv.org/abs/1511.06434) (look at the box on page 3, and at image on page 4). What differences do you see? What do the two networks have in common?

In [ ]:
def build_generator():
    
    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(n_channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    return model

Now we instantiate the discriminator and we compile it. We use Adam solver, but not with standard parameters! You can find the right parameters in the original paper, or, more easily, in [this post...](https://medium.com/@jonathan_hui/gan-dcgan-deep-convolutional-generative-adversarial-networks-df855c438f)

With respect to previous notebooks, there is a different loss function... can you guess why?

In [ ]:
# TODO assign the correct values to the following variables
adam_learning_rate =
momentum_beta1     = 

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(adam_learning_rate, momentum_beta1),
                      metrics=['accuracy'])

Now we need to instantiate the generator. After that, we create a stacked model, which puts generator and discriminator together. To do this, we use the `Model API`, instead of the `Sequential` API we have used until now.

The stacked model has:

- an input layer `z` which takes the random noise vector
- the generator, which takes `z` as input and returns a generated image `img`
- the discriminator, which takes `img` as input and outputs a probability `valid`. The discriminator will not be trained in this phase (we have a separate training for it, see below!)
- `valid` will also be the output of our `combined` model. You can see that we build it by calling the Model constructor which takes two simple arguments, the input and the output.

In [ ]:
# Build the generator
generator = build_generator()

# The generator takes noise as input and generates imgs
z = Input(shape=(latent_dim,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(inputs=z, outputs=valid)
combined.compile(loss='binary_crossentropy', optimizer=Adam(adam_learning_rate, momentum_beta1))

And we are ready to train the networks!

This training is different from the usual `fit` or `fit_generator` functions we were using in the previous notebooks. Here, we first train the discriminator on **one batch of real images**, then on **one batch of generated images**. After that, we train the stack model (where we do not train the generator part anymore) on a set of random vectors called `noise`. We then repeat these three steps as much as we need. Fill the only missing variable (again, look at [this post...](https://medium.com/@jonathan_hui/gan-dcgan-deep-convolutional-generative-adversarial-networks-df855c438f)), understand what the code is doing (especially where you find a *`TODO`*) and run it! You might be interested in reading how `train_on_batch` is defined, [here](https://keras.io/models/model/#train_on_batch)... and disregard any warning message!

In [ ]:
steps=4001
save_interval=50
#TODO set the correct batch size!
batch_size=


# TODO can you understand what these two variables are needed for?
# Adversarial ground truths
valid = np.ones((batch_size, 1))
fake  = np.zeros((batch_size, 1))


for step in range(steps):

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random batch of images
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    # Sample noise and generate a batch of new images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    # This is just an indication of the average loss!
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------

    # Train the generator (wants discriminator to mistake images as real)
    # TODO: why do we use valid?
    g_loss = combined.train_on_batch(noise, valid)


    # If at save interval => save generated image samples
    if step % save_interval == 0:    
        # Plot the progress
        titlestring="Step {} [D loss: {}, acc.: {:.2f}%] [G loss: {}]".format(step, d_loss[0], 100*d_loss[1], g_loss)
        #print(titlestring)
        noise = np.random.normal(0, 1, (20, latent_dim))
        gen_imgs = generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        plt.figure(figsize=(100,10))
        for j in range(10):
            plt.subplot(1, 10, j+1)
            plt.imshow(gen_imgs[j, :,:,0], cmap='gray')
            plt.axis('off')
        #fig.savefig("images/mnist_%d.png" % epoch)
        plt.suptitle(titlestring, fontsize=80)
        plt.show()